In [281]:
!pip install requests
!pip install beautifulsoup
from bs4 import BeautifulSoup, Comment
import pandas as pd
import sys
import requests
import re


You are using pip version 19.0, however version 21.3.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
  Using cached https://files.pythonhosted.org/packages/40/f2/6c9f2f3e696ee6a1fb0e4d7850617e224ed2b0b1e872110abffeca2a09d4/BeautifulSoup-3.2.2.tar.gz
    Complete output from command python setup.py egg_info:
    Traceback (most recent call last):
      File "<string>", line 1, in <module>
      File "/tmp/pip-install-b6u4dfrf/beautifulsoup/setup.py", line 3
        "You're trying to run a very old release of Beautiful Soup under Python 3. This will not work."<>"Please use Beautiful Soup 4, available through the pip package 'beautifulsoup4'."
                                                                                                        ^
    SyntaxError: invalid syntax
    
    ----------------------------------------
Command "python setup.py egg_info" failed with error code 1 in /tmp/pip-install-b6u4dfrf/beautifulsoup/
You are using pi

In [292]:
url_temp = "https://www.basketball-reference.com/friv/teammates_and_opponents.fcgi?pid={}&type=t"

In [520]:
all_players = pd.read_csv("BBRefIDs.csv")
all_players = all_players["BBRefID"] #get all the ids
player_names = pd.read_csv("BBRefIDs.csv")['BBRefName']

In [521]:
all_players = all_players.to_frame()

In [522]:
all_players["Name"] = player_names
all_players["value"] = 9999
all_players["previous"] = "null"
all_players["visited"] = False
all_players = all_players.set_index("BBRefID")
permanent_all_players = all_players.copy()

In [523]:
def getTeammates(idP):
    url = url_temp.format(idP)
#     print(url)
    data = requests.get(url)
    with open("data.html", "w+") as f:
        f.write(data.text)
    with open("data.html") as f:
        page = f.read();
        soup = BeautifulSoup(page, "html.parser");      
        x = soup.find_all('tr', {"class":"thead"})
        y = soup.find_all('tr', {"class":"over_header"})
        for a in x:
            a.decompose()
        for a in y:
            a.decompose()
        teammates = str(soup.find("table",{'id':"teammates-and-opponents"}))
        teammate_ids = [m.start() for m in re.finditer('/players/', teammates)]
        result = re.findall('/players/(.*)/(.*).html', teammates)
        teammates_data = []
        for r in result:
            teammates_data.append(r[1])
    return teammates_data

In [524]:
def recAdd(playerID, count):
#     print("adding")
    teammateList = getTeammates(playerID)
    if((last in teammateList) == True):
        found = True;
        all_players.at[last,'value'] = count
        all_players.at[last,'previous'] = playerID
        print(count)
        print("found it!")
        return True
    for teammate in teammateList:
        if((teammate in all_players.index) == False):
            return False
        if(count<all_players.loc[teammate]['value']): # if new shortest path, update previous
            all_players.at[teammate,'value'] = count #update the path count
            all_players.at[teammate,'previous'] = playerID #update previous
        if(all_players.loc[teammate]['visited'] == False): # if we havent visited this neighbor, then do the same shit
#             print("appending "+teammate)
            visit.append(teammate)
            all_players.at[teammate,'visited'] = True
            
            

In [525]:
getTeammates("bealbr01")

['walljo01',
 'gortama01',
 'porteot01',
 'oubreke01',
 'satorto01',
 'morrima02',
 'hilarne01',
 'templga01',
 'mahinia01',
 'serapke01',
 'arizatr01',
 'webstma02',
 'bertada01',
 'smithja02',
 'bryanth01',
 'browntr01',
 'booketr01',
 'hachiru01',
 'goodedr01',
 'sessira01',
 'smithis01',
 'netora01',
 'bongais01',
 'avdijde01',
 'scottmi01',
 'millean02',
 'meeksjo01',
 'greenje02',
 'mathega01',
 'piercpa01',
 'lopezro01',
 'humphkr01',
 'butlera01',
 'fraziti01',
 'gaffoda01',
 'westbru01',
 'wagnemo01',
 'veselja01',
 'singlch01',
 'okafoem01',
 'burketr01',
 'dudleja01',
 'lenal01',
 'randlch01',
 'mcraejo01',
 'gillan01',
 'harrial01',
 'caldwke01',
 'priceaj01',
 'thornma01',
 'dekkesa01',
 'bogdabo02',
 'kuzmaky01',
 'crawfjo02',
 'blairde01',
 'kispeco01',
 'jennibr01',
 'harremo01',
 'thomais02',
 'mcclesh01',
 'holidaa01',
 'dinwisp01',
 'riverau01',
 'portibo01',
 'robinje01',
 'nichoan01',
 'paytoga02',
 'parkeja01',
 'nealga01',
 'winstca01',
 'martica01',
 'schofad01'

In [549]:
first = "zelleco01"
last = "mitchdo01"
all_players = permanent_all_players.copy()
found = False
visit = [];

In [550]:
all_players.at[first,"value"] = 0;
visit.append(first)
print("Looking for path")
while(len(visit)>0 and found == False):
    player_to_test = visit[0];
    visit.remove(player_to_test)
    x = recAdd(player_to_test,all_players.loc[player_to_test]['value']+1)
    if(x == True):
        break

previous = last
current = all_players.loc[last]["previous"]

while(True):
    print(all_players.loc[previous]["Name"] + " played with " + all_players.loc[current]["Name"])
    previous = current
    current = all_players.loc[previous]["previous"]
    if(current==first):
        print("and "+all_players.loc[previous]["Name"] + " played with " + all_players.loc[current]["Name"])
        break

Looking for path
3
found it!
Donovan Mitchell played with Alec Burks
Alec Burks played with Kemba Walker
and Kemba Walker played with Cody Zeller
